In [13]:
import json
import requests
import time
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv()) # retrieve all env vars

fmp_api_key = os.getenv("FMP_API_KEY")

client = OpenAI()



### Steps to create this assistant

1. Define financial `functions`

2. Define `Assistant`

3. Create a `Thread`

4. Add `Messages`

5. `Run` the Assistant

## 1. Functions


In [14]:
def get_income_statement(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())    

def get_financial_growth(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())      

In [15]:
# Storing functions in a dictionary

available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_financial_ratios,
    "get_financial_growth": get_financial_growth
}

## 2. Assistant

In [30]:
# def run_assistant(user_message: str):
assistant = client.beta.assistants.create(
    instructions="Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends",
    model="gpt-3.5-turbo-1106",
    tools=[
        # income statement function
        {
            "type": "function",
            "function": {
                    "name": "get_income_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # balance sheet
        {
            "type": "function",
            "function": {
                    "name": "get_balance_sheet",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # cash flow statement
        {
            "type": "function",
            "function": {
                    "name": "get_cash_flow_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # key metrics
        {
            "type": "function",
            "function": {
                    "name": "get_key_metrics",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # financial ratios
        {
            "type": "function",
            "function": {
                    "name": "get_financial_ratios",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # financial growth
        {
            "type": "function",
            "function": {
                    "name": "get_financial_growth",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },
    ]
)

dict(assistant)

{'id': 'asst_WiYzd4bALikEx2wzVrdtyhsq',
 'created_at': 1704823769,
 'description': None,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(name='get_balance_sheet', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(nam

## 3. Thread

In [31]:
thread = client.beta.threads.create()

dict(thread)

{'id': 'thread_3bUjhSUtGViKdFHkDq1lGwjH',
 'created_at': 1704823777,
 'metadata': {},
 'object': 'thread'}

## 4. Add Message

In [28]:
# Adding initial user message
user_message = "Give an overview of income statement"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

dict(message)


{'id': 'msg_NF3iXDGqVaHCJl911T3Brbq3',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Give an overview of income statement'), type='text')],
 'created_at': 1704823734,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_o2sLK8rvBoYlzTTKWx8ZE891'}

## 5. Run 

In [32]:
# 5.1 Start a Run

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

dict(run)


{'id': 'run_DgGicZ3x3RaAoB3HwfSKPSZw',
 'assistant_id': 'asst_WiYzd4bALikEx2wzVrdtyhsq',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704823806,
 'expires_at': 1704824406,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_3bUjhSUtGViKdFHkDq1lGwjH',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolAssistantToolsFunction(fu